In [15]:
import numpy as np
from scipy import signal as sg
from time import time

In [16]:
#helper function
def formHi(v,N1):
    R=np.array([v]).T
    
    for i in range(N1-1):
        v=np.insert(v,0,v[-1])
        v=np.delete(v,-1)
        R=np.hstack((R,np.array([v]).T))
    return R

#helper function
def formH(Hlist,M1,N3):
    
    vertical=Hlist[0]
    for i in range(1,len(Hlist)):
        vertical=np.vstack((vertical,Hlist[i]))
    
    R=vertical.copy()
    
    for _ in range(M1-1):
        for i in range(N3):
            vertical=np.insert(vertical,0,vertical[-1-i],axis=0)
        for i in range(N3):
            vertical=np.delete(vertical,-1,axis=0)
        R=np.hstack((R,vertical))
    
    return R
#helper function
def formF(A):
    R=np.array([])
    for row in A:
        R=np.append(R,row)
    return R.reshape(-1,1)

def toeplitzConvolution(A,K):
    """
    Returns 2D convolution of matrix A with kernel K by forming doubly block Toeplitz matrix
    """
    M1,N1=A.shape
    M2,N2=K.shape
    M3=M1+M2-1
    N3=N1+N2-1
    
    K_padded=np.zeros((M3,N3))
    K_padded[:M2,:N2]=K
    
    f=formF(A)
    
    Hlist=[]
    for row in K_padded:
        Hlist.append(formHi(row,N1))
    
    H=formH(Hlist,M1,N3)
    
    return np.dot(H,f).reshape(M3,N3)

def circularConvolution(v,k):
    """
    Forms circulant matrix C from vector v and returns convolution of vector v 
    with kernel k, along with matrix C
    """
    C=np.array([])
    C=v.reshape(-1,1)
    for i in range(v.size-k.size):
        k=np.append(k,0)
        
    for i in range(v.size-1):
        v=np.insert(v,0,v[-1])
        v=np.delete(v,-1)
        C=np.hstack((C,np.array([v]).T))
    return np.dot(C,k),C

def circulantMatrixEig(C):
    """
    Return eigen vectors of circulant matrix C as columns of result matrix
    """
    n=C.shape[0]
    F=np.zeros((n,n),dtype='complex')
    
    for j in range(C.shape[0]):
        for k in range(C.shape[0]):
            F[j,k]=np.complex(np.cos(2*np.pi*j*k/n),np.sin(2*np.pi*j*k/n))
    return F

For periodic input x=[1 8 3 2 5] and system response h=[3 5 2 4 1]:

In [17]:
x,C=circularConvolution(np.array([1,8,3,2,5]),np.array([3,5,2,4,1]))
print('x*h =',x)
print('C = \n',C)

x*h = [52 50 73 46 64]
C = 
 [[1 5 2 3 8]
 [8 1 5 2 3]
 [3 8 1 5 2]
 [2 3 8 1 5]
 [5 2 3 8 1]]


For every Circulant matrix C, one eigen vector is always [1,1,...,1] and corresponding eigen value is sum of all elements in one row of matrix C

In [18]:
print('Eigen vector:',np.ones(C.shape[0]))
print('Corresponding eigen value:',np.sum(C[0]))

Eigen vector: [1. 1. 1. 1. 1.]
Corresponding eigen value: 19


For Circulant matrices C1 and C2 of the same shape,all eigen vectors are the same:

In [19]:
x1,C1=circularConvolution(np.random.randint(0,100,4),np.array([1,2,3]))
x2,C2=circularConvolution(np.random.randint(0,100,4),np.array([1,2,3]))

vectors1=np.linalg.eig(C1)[1]
vectors2=np.linalg.eig(C2)[1]
eig_vectors_diff=[]
for i in range(vectors1.shape[1]):
    eig_vectors_diff.append(sum(vectors1[:,i]-vectors2[:,i]))
    
print(np.sum(eig_vectors_diff))

(4.000000000000002+5.33340727540123e-17j)


In [20]:
a=np.random.randint(0,100,(50,50))
b=np.random.randint(0,10,(3,3))
t0=time()
y=sg.convolve2d(a,b)
t1=time()
x=toeplitzConvolution(a,b)
t2=time()

#print(sum(x-y))
print('signal.convolve2d:',t1-t0)
print('toeplitz convolution:',-t1+t2)


signal.convolve2d: 0.0004444122314453125
toeplitz convolution: 1.5320701599121094


In [21]:
F=circulantMatrixEig(C1)
#normiranje kolona je potrebno naknadno odraditi
#for i in range(F.shape[0]):
#    F[:,i]=F[:,i]/np.linalg.norm(F[:,i])

Eigen vectors for all circulant matrices dim=(4,4):

In [22]:
for i in range(F.shape[0]):
    print(F[:,i])

[1.+0.j 1.+0.j 1.+0.j 1.+0.j]
[ 1.0000000e+00+0.0000000e+00j  6.1232340e-17+1.0000000e+00j
 -1.0000000e+00+1.2246468e-16j -1.8369702e-16-1.0000000e+00j]
[ 1.+0.0000000e+00j -1.+1.2246468e-16j  1.-2.4492936e-16j
 -1.+3.6739404e-16j]
[ 1.0000000e+00+0.0000000e+00j -1.8369702e-16-1.0000000e+00j
 -1.0000000e+00+3.6739404e-16j  5.5109106e-16+1.0000000e+00j]


In [23]:
G=circulantMatrixEig(C2)
for i in range(G.shape[0]):
    G[:,i]=G[:,i]/np.linalg.norm(G[:,i])
    
for i in range(G.shape[0]):
    print(G[:,i])

[0.5+0.j 0.5+0.j 0.5+0.j 0.5+0.j]
[ 5.00000000e-01+0.000000e+00j  3.06161700e-17+5.000000e-01j
 -5.00000000e-01+6.123234e-17j -9.18485099e-17-5.000000e-01j]
[ 0.5+0.0000000e+00j -0.5+6.1232340e-17j  0.5-1.2246468e-16j
 -0.5+1.8369702e-16j]
[ 5.00000000e-01+0.0000000e+00j -9.18485099e-17-5.0000000e-01j
 -5.00000000e-01+1.8369702e-16j  2.75545530e-16+5.0000000e-01j]


We get eigen values by multiplying F with first row of matrix C

In [24]:
eigenvals=np.dot(F,C1[0,:])
eigenvals

array([184.+0.0000000e+00j, -50.+2.0000000e+01j,  16.-2.4492936e-16j,
       -50.-2.0000000e+01j])

In [25]:
np.linalg.eigvals(C1)

array([184. +0.j,  16. +0.j, -50.+20.j, -50.-20.j])